In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/06/22 11:28:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df_green = spark.read.parquet('data/raw/green/*/*')

In [3]:
df_green.registerTempTable('green')

In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue.show()

+-------------------+----+-----------------+--------------+
|               hour|zone|           amount|number_records|
+-------------------+----+-----------------+--------------+
|2021-10-01 06:00:00| 117|            93.08|             2|
|2021-10-01 13:00:00| 131|             18.1|             1|
|2021-10-01 15:00:00| 179|             15.3|             1|
|2021-10-02 02:00:00| 255|             6.96|             1|
|2021-10-02 15:00:00| 243|           101.21|             3|
|2021-10-02 16:00:00|  31|           125.65|             2|
|2021-10-02 22:00:00| 152|             18.8|             2|
|2021-10-02 23:00:00| 129|             78.4|             3|
|2021-10-03 00:00:00|  75|            83.28|             4|
|2021-10-03 08:00:00| 127|            42.06|             1|
|2021-10-03 11:00:00|  83|             51.8|             2|
|2021-10-03 14:00:00|  97|            50.75|             4|
|2021-10-03 15:00:00| 166|            89.07|             5|
|2021-10-04 09:00:00|  49|72.27000000000

In [6]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [7]:
df_yellow = spark.read.parquet('data/raw/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [8]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [9]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [10]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [11]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [12]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [13]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [14]:
df_join = spark.read.parquet('data/report/revenue/total')

In [15]:
df_join

DataFrame[hour: timestamp, zone: bigint, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [16]:
df_zones = spark.read.parquet('zones/')

AnalysisException: Path does not exist: file:/home/abura/data-engineering-zoomcamp/week_5_batch_processing/code/zones;

In [57]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [62]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')